In [ ]:
%matplotlib qt5
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
Alpha=0.1 #学习速率
Beta=0.9 #对未来奖励的衰减度
Epsilon_start=1#选择随机动作的概率 Epsilon
Epsilon_stop=0.01
decay_rate=0.00001
N_Mesh=10
N_States=100#假设网格为5*5
Actions=['left','right','up','down']
Obstacle=[3,5,16,21,23,27,29,31,33,34,56,57,66,67,76,78,81,86,96,98]
#——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
def Caculate_coordinate(State):
    X_Axis=0.5+(State-1)%N_Mesh
    Y_Axis=0.5+((State-1)//N_Mesh)
    return X_Axis,Y_Axis
def Initial_Q_Table (N_States,Actions):#以状态数和动作为变量，返回Q表
    Q_Table=np.zeros([N_States,len(Actions)])#初始化Q_表
    Q_Table=pd.DataFrame(Q_Table,index=np.arange(1,N_States+1),columns=Actions)#Q_表定义行列标签
    print ('初始化Q表完成') 
    return Q_Table
#——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
def Select_Action(State,Q_Table,ep):#根据目前的状态选择动作,返回执行动作
    Epsilon=Epsilon_stop+(Epsilon_start-Epsilon_stop)*np.exp(-decay_rate*ep)
    States_Actions=Q_Table.loc[[State],:]#选择目前状态下，取出Q表的目前的状态行,state选择为数字
    if(np.random.rand()<Epsilon or np.all(States_Actions==[0])):#其实np.all(States_Action==[0])也行
        Execute_Action=np.random.choice(Actions)
    else:
        #Q_T=Q_Table.T
        #G=(Q_Table.T).loc[:,[State]]
        Execute_Action=np.array(pd.DataFrame.idxmax((Q_Table.T).loc[:,[State]]))[0]#这里真心累，有毒
    return Execute_Action
#———————————————————————————————————————————————————————————————————————————————————————————————————————————
def Next_State(State,Execute_Action):#根据目前状态个选择的动作,返回即时奖励和新状态
    if(Execute_Action=='left'):
        #目标在右上角，如果Execute_Action为‘left’，不可能到达目标
        if(State%N_Mesh==1):#判断是不是在网格最左边
            State=State
            R=-100
        elif((State-1) in Obstacle):
            State=State-1
            R=-100
        else:
            State=State-1
            R=1
    elif(Execute_Action=='right'):
        if(State==N_States-1):
            State='End'
            R=100
        elif(State%N_Mesh==0):
            State=State
            R=-100
        elif((State+1) in Obstacle):
            State=State+1
            R=-100
        else:
            State=State+1
            R=1
    elif(Execute_Action=='up'):
        if(State==N_States-N_Mesh):
            State='End'
            R=100
        elif(N_States+1-N_Mesh<=State<=N_States-1):
            State=State
            R=-100
        elif((State+10) in Obstacle):
            State=State+10
            R=-100
        else:
            State=State+N_Mesh
            R=1
    else:
        if(0<=State<=N_Mesh):
            State=State
            R=-100
        elif((State-10) in Obstacle):
            State=State-10
            R=-100
        else:
            R=0
            State=State-10
    return State,R
        
#————————————————————————————————————————————————————————————————————————————————————————————
def Run_Function():
    #——————————————————————————————————————————————————画图程序部分
    
    
    #——————————————————————————————————————————————————
    Q_Table=Initial_Q_Table (N_States,Actions)
    Step=0 
    for episode in np.arange(1000) :#跑20回合
        if(episode%300==0):
            print('Q_Table,%d'%episode)
            Q_Table
#         fig=plt.figure()
#         ax=fig.gca()
#         ax.set(xlim=[0, N_Mesh], ylim=[0, N_Mesh])
#         ax.set_xticks(np.arange(0,(N_Mesh+1)))
#         ax.set_yticks(np.arange(0,(N_Mesh+1)))
#         plt.grid()
        print('episode is %d' % episode)
        if(np.random.rand()<0.4):
            State_=1 #定义起点 
            #print("Normal_start:%d"%State_)
        else:
            State_=np.random.randint(low=5, high=100)
            while(State_ in Obstacle):
                State_=np.random.randint(low=5, high=100)
            #print("Random_start:%d"%State_)
        Is_End=False
        while not Is_End:
            A=Select_Action(State_,Q_Table,Step)
            Next_S,R=Next_State(State_,A)
            print(Next_S)
            Q_Table_S_A=Q_Table.loc[[State_],[A]]
            if Next_S!='End':
                D_T=np.array((Q_Table.loc[[Next_S],:]).T)
                Q_Target=R+Beta*D_T.max() #Next_S状态中的可选动作中的最大Q值动作
                Step+=1
            else:
                Q_Target=R
                Step+=1
                Is_End=True #目的是结束循
#————————————————————————————————————————————————————————————————————————————画图程序
#             X_State,Y_State=Caculate_coordinate(State_)
#             if(Next_S)=='End':
#                 H=N_States
#             else:
#                 H=Next_S
#             X_Next_State,Y_Next_State=Caculate_coordinate(H)                      
#             if(Step==0):
#                 plt.scatter(X_State,Y_State)  
#             else:
#                 plt.scatter(X_State,Y_State) 
#                 plt.scatter(X_Next_State,Y_Next_State)
#                 plt.plot([X_State,X_Next_State],[Y_State,Y_Next_State])
#————————————————————————————————————————————————————————————————————————————
            Q_Table.loc[[State_],[A]]+=Alpha*(Q_Target-Q_Table_S_A)
            State_=Next_S
        print(Step)
        print(Epsilon_stop+(Epsilon_start-Epsilon_stop)*np.exp(-decay_rate*Step))
        #plt.show()
        #print(Q_Table)
    return Q_Table
#————————————————————————————————————————————————————————————————————————————————————————————————————
Q__Table=Run_Function()
Q__Table
Final_Q_Table=np.array(Q__Table)
#—————————————————————————————————————————————————————————————————————————————————————————————————————
def Plot_Final_Graph(Q_TABLE):
    f=open('A.txt','w+')
    f.write(str(Q_TABLE))
    f.close()
    fig=plt.figure()
    ax=fig.gca()
    ax.set(xlim=[0, N_Mesh], ylim=[0, N_Mesh])
    ax.set_xticks(np.arange(0,(N_Mesh+1)))
    ax.set_yticks(np.arange(0,(N_Mesh+1)))
    plt.grid()
    State_Start=1
    State=State_Start
    Step=1
    while(State!=N_States):
        Action=np.argmax((Q_TABLE[State-1]))
        if(Action==0):
             Next_State=State-1
        elif(Action==1):
             Next_State=State+1
        elif(Action==2):
             Next_State=State+N_Mesh
        else:
             Next_State=State-N_Mesh
        Step+=1
        print(Next_State)
        X_State,Y_State=Caculate_coordinate(State)
        X_Next_State,Y_Next_State=Caculate_coordinate(Next_State)
        if(Step==0):
            plt.scatter(X_State,Y_State)  
        else:
            plt.scatter(X_State,Y_State) 
            plt.scatter(X_Next_State,Y_Next_State)
            plt.plot([X_State,X_Next_State],[Y_State,Y_Next_State])
        for J in np.arange(len(Obstacle)):
            plt.scatter((Caculate_coordinate(Obstacle[J]))[0],(Caculate_coordinate(Obstacle[J]))[1],marker="x")
        #print(X_State,Y_State)
        #print(X_Next_State,Y_Next_State)
        State=Next_State
    plt.show()
#——————————————————————————————————————————————————————————————————————————————————————————————————————————
Plot_Final_Graph(Final_Q_Table)

In [6]:
%matplotlib qt5
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
Alpha=0.1 #学习速率
Beta=0.9 #对未来奖励的衰减度
Epsilon_start=1#选择随机动作的概率 Epsilon
Epsilon_stop=0.01
decay_rate=0.00001
N_Mesh=10
N_States=100#假设网格为5*5
Actions=['left','right','up','down']
Obstacle=[3,5,13,16,21,23,27,29,31,33,34,43,53,56,57,63,66,67,76,78,81,86,96,98]
#——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
def Caculate_coordinate(State):
    X_Axis=0.5+(State-1)%N_Mesh
    Y_Axis=0.5+((State-1)//N_Mesh)
    return X_Axis,Y_Axis
def Initial_Q_Table (N_States,Actions):#以状态数和动作为变量，返回Q表
    Q_Table=np.zeros([N_States,len(Actions)])#初始化Q_表
    Q_Table=pd.DataFrame(Q_Table,index=np.arange(1,N_States+1),columns=Actions)#Q_表定义行列标签
    print ('初始化Q表完成') 
    return Q_Table
#——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
def Select_Action(State,Q_Table,ep):#根据目前的状态选择动作,返回执行动作
    Epsilon=Epsilon_stop+(Epsilon_start-Epsilon_stop)*np.exp(-decay_rate*ep)
    States_Actions=Q_Table.loc[[State],:]#选择目前状态下，取出Q表的目前的状态行,state选择为数字
    if(np.random.rand()<Epsilon or np.all(States_Actions==[0])):#其实np.all(States_Action==[0])也行
        Execute_Action=np.random.choice(Actions)
    else:
        #Q_T=Q_Table.T
        #G=(Q_Table.T).loc[:,[State]]
        Execute_Action=np.array(pd.DataFrame.idxmax((Q_Table.T).loc[:,[State]]))[0]#这里真心累，有毒
    return Execute_Action
#———————————————————————————————————————————————————————————————————————————————————————————————————————————
def Next_State(State,Execute_Action):#根据目前状态个选择的动作,返回即时奖励和新状态
    if(Execute_Action=='left'):
        #目标在右上角，如果Execute_Action为‘left’，不可能到达目标
        if(State%N_Mesh==1):#判断是不是在网格最左边
            State=State
            R=-10000
        elif((State+1) in Obstacle):
            State=State-1
            R=-10000
        else:
            State=State-1
            R=0
    elif(Execute_Action=='right'):
        if(State==N_States-1):
            State='End'
            R=10
        elif(State%N_Mesh==0):
            State=State
            R=-10000
        elif((State+1) in Obstacle):
            State=State+1
            R=-10000
        else:
            State=State+1
            R=0.002
    elif(Execute_Action=='up'):
        if(State==N_States-N_Mesh):
            State='End'
            R=10
        elif(N_States+1-N_Mesh<=State<=N_States-1):
            State=State
            R=-10000
        elif((State+10) in Obstacle):
            State=State+10
            R=-10000
        else:
            State=State+N_Mesh
            R=0.002
    else:
        if(0<=State<=N_Mesh):
            State=State
            R=-10000
        elif((State-10) in Obstacle):
            State=State-10
            R=-10000
        else:
            R=0
            State=State-10
    return State,R
        
#————————————————————————————————————————————————————————————————————————————————————————————
def Run_Function():
    #——————————————————————————————————————————————————画图程序部分
    
    
    #——————————————————————————————————————————————————
    Q_Table=Initial_Q_Table (N_States,Actions)
    Step=0 
    for episode in np.arange(700) :#跑20回合
        if(episode%300==0):
            print('Q_Table,%d'%episode)
            Q_Table
#         fig=plt.figure()
#         ax=fig.gca()
#         ax.set(xlim=[0, N_Mesh], ylim=[0, N_Mesh])
#         ax.set_xticks(np.arange(0,(N_Mesh+1)))
#         ax.set_yticks(np.arange(0,(N_Mesh+1)))
#         plt.grid()
        print('episode is %d' % episode)
        if(np.random.rand()<0.4):
            State_=1 #定义起点 
            print("Normal_start:%d"%State_)
        else:
            State_=np.random.randint(low=5, high=100)
            print("Random_start:%d"%State_)
        Is_End=False
        while not Is_End:
            A=Select_Action(State_,Q_Table,Step)
            Next_S,R=Next_State(State_,A)
            #print(Next_S)
            Q_Table_S_A=Q_Table.loc[[State_],[A]]
            if Next_S!='End':
                D_T=np.array((Q_Table.loc[[Next_S],:]).T)
                Q_Target=R+Beta*D_T.max() #Next_S状态中的可选动作中的最大Q值动作
                Step+=1
            else:
                Q_Target=R
                Step+=1
                Is_End=True #目的是结束循
#————————————————————————————————————————————————————————————————————————————画图程序
#             X_State,Y_State=Caculate_coordinate(State_)
#             if(Next_S)=='End':
#                 H=N_States
#             else:
#                 H=Next_S
#             X_Next_State,Y_Next_State=Caculate_coordinate(H)                      
#             if(Step==0):
#                 plt.scatter(X_State,Y_State)  
#             else:
#                 plt.scatter(X_State,Y_State) 
#                 plt.scatter(X_Next_State,Y_Next_State)
#                 plt.plot([X_State,X_Next_State],[Y_State,Y_Next_State])
#————————————————————————————————————————————————————————————————————————————
            Q_Table.loc[[State_],[A]]+=Alpha*(Q_Target-Q_Table_S_A)
            State_=Next_S
        print(Step)
        print(Epsilon_stop+(Epsilon_start-Epsilon_stop)*np.exp(-decay_rate*Step))
        #plt.show()
        #print(Q_Table)
    return Q_Table
#————————————————————————————————————————————————————————————————————————————————————————————————————
Q__Table=Run_Function()
Q__Table
Final_Q_Table=np.array(Q__Table)
#—————————————————————————————————————————————————————————————————————————————————————————————————————
def Plot_Final_Graph(Q_TABLE):
    fig=plt.figure()
    ax=fig.gca()
    ax.set(xlim=[0, N_Mesh], ylim=[0, N_Mesh])
    ax.set_xticks(np.arange(0,(N_Mesh+1)))
    ax.set_yticks(np.arange(0,(N_Mesh+1)))
    plt.grid()
    State_Start=1
    State=State_Start
    Step=1
    while(State!=N_States):
        Action=np.argmax((Q_TABLE[State-1]))
        if(Action==0):
             Next_State=State-1
        elif(Action==1):
             Next_State=State+1
        elif(Action==2):
             Next_State=State+N_Mesh
        else:
             Next_State=State-N_Mesh
        Step+=1
        print(Next_State)
        X_State,Y_State=Caculate_coordinate(State)
        X_Next_State,Y_Next_State=Caculate_coordinate(Next_State)
        if(Step==0):
            plt.scatter(X_State,Y_State)  
        else:
            plt.scatter(X_State,Y_State) 
            plt.scatter(X_Next_State,Y_Next_State)
            plt.plot([X_State,X_Next_State],[Y_State,Y_Next_State])
        for J in np.arange(len(Obstacle)):
            plt.scatter((Caculate_coordinate(Obstacle[J]))[0],(Caculate_coordinate(Obstacle[J]))[1],marker="x")
        print(X_State,Y_State)
        print(X_Next_State,Y_Next_State)
        State=Next_State
    plt.show()
#——————————————————————————————————————————————————————————————————————————————————————————————————————————
Plot_Final_Graph(Final_Q_Table)

初始化Q表完成
Q_Table,0
episode is 0
Normal_start:1
853
0.991591214456
episode is 1
Random_start:24
1808
0.982261637992
episode is 2
Random_start:31
1907
0.98129957527
episode is 3
Random_start:81
1941
0.98096938955
episode is 4
Normal_start:1
2483
0.975720971519
episode is 5
Normal_start:1
3306
0.967805703927
episode is 6
Random_start:46
3439
0.966532669096
episode is 7
Random_start:17
3991
0.961267154952
episode is 8
Random_start:26
4188
0.959395003332
episode is 9
Normal_start:1
4634
0.955170130088
episode is 10
Random_start:84
4689
0.954650429447
episode is 11
Normal_start:1
4827
0.953347710937
episode is 12
Random_start:64
4863
0.953008166882
episode is 13
Normal_start:1
4958
0.952112734521
episode is 14
Random_start:69
4973
0.95197142821
episode is 15
Normal_start:1
5710
0.945054618536
episode is 16
Random_start:56
5728
0.944886323851
episode is 17
Normal_start:1
6008
0.942272303481
episode is 18
Normal_start:1
6338
0.939200875523
episode is 19
Random_start:14
6456
0.938105065145
episo

25085
0.780357692831
episode is 164
Random_start:42
25216
0.77934918497
episode is 165
Random_start:41
25343
0.778372731685
episode is 166
Random_start:23
25391
0.778004001276
episode is 167
Normal_start:1
25491
0.777236381149
episode is 168
Random_start:40
25574
0.776599839154
episode is 169
Random_start:82
25607
0.776346902944
episode is 170
Random_start:62
25703
0.775611562936
episode is 171
Normal_start:1
25824
0.774685733185
episode is 172
Random_start:37
25893
0.774158282021
episode is 173
Normal_start:1
25972
0.773554835371
episode is 174
Normal_start:1
26060
0.772883202677
episode is 175
Normal_start:1
26143
0.772250272322
episode is 176
Random_start:52
26225
0.771625483297
episode is 177
Normal_start:1
26294
0.771100142977
episode is 178
Normal_start:1
26435
0.770027747991
episode is 179
Random_start:77
26535
0.76926810013
episode is 180
Random_start:75
26549
0.769161810037
episode is 181
Random_start:49
26679
0.768175540898
episode is 182
Random_start:88
26685
0.76813005173
e

35871
0.70159114019
episode is 325
Normal_start:1
35929
0.701190133632
episode is 326
Normal_start:1
35999
0.70070646984
episode is 327
Normal_start:1
36134
0.699774645229
episode is 328
Random_start:83
36180
0.699457421859
episode is 329
Random_start:79
36185
0.69942294985
episode is 330
Random_start:33
36215
0.699216153986
episode is 331
Normal_start:1
36244
0.69901631028
episode is 332
Random_start:45
36283
0.698747646312
episode is 333
Random_start:79
36305
0.698596138496
episode is 334
Random_start:20
36339
0.698362055606
episode is 335
Random_start:85
36350
0.698286339944
episode is 336
Random_start:60
36392
0.69799732038
episode is 337
Random_start:44
36448
0.697612149738
episode is 338
Random_start:43
36562
0.696828718528
episode is 339
Random_start:56
36583
0.696684499641
episode is 340
Random_start:79
36592
0.696622700817
episode is 341
Random_start:93
36673
0.696066761615
episode is 342
Random_start:35
36731
0.695668958267
episode is 343
Normal_start:1
36808
0.695141196384
e

43866
0.648451008278
episode is 486
Random_start:15
43892
0.648285032594
episode is 487
Random_start:18
43921
0.648099956771
episode is 488
Random_start:30
43937
0.647997868946
episode is 489
Random_start:63
44002
0.647583305079
episode is 490
Normal_start:1
44057
0.647232730678
episode is 491
Normal_start:1
44138
0.646716781154
episode is 492
Random_start:87
44156
0.646602182447
episode is 493
Normal_start:1
44220
0.646194887399
episode is 494
Random_start:67
44230
0.646131271091
episode is 495
Random_start:33
44253
0.645984977723
episode is 496
Random_start:76
44283
0.645794210846
episode is 497
Random_start:72
44376
0.645203197094
episode is 498
Normal_start:1
44434
0.64483488606
episode is 499
Normal_start:1
44570
0.643972097444
episode is 500
Normal_start:1
44615
0.643686874181
episode is 501
Normal_start:1
44672
0.643325775585
episode is 502
Normal_start:1
44737
0.642914247592
episode is 503
Random_start:46
44755
0.64280033328
episode is 504
Normal_start:1
44819
0.642395470637
ep

episode is 646
Random_start:92
50434
0.607864980373
episode is 647
Normal_start:1
50453
0.607751396818
episode is 648
Random_start:64
50496
0.607494418972
episode is 649
Normal_start:1
50529
0.607297278343
episode is 650
Random_start:72
50595
0.606903192202
episode is 651
Normal_start:1
50641
0.606628679877
episode is 652
Random_start:41
50672
0.606443753651
episode is 653
Random_start:37
50701
0.60627081004
episode is 654
Random_start:37
50722
0.606145606317
episode is 655
Random_start:36
50756
0.605942951264
episode is 656
Random_start:81
50802
0.605668880548
episode is 657
Normal_start:1
50827
0.605519981941
episode is 658
Random_start:97
50836
0.605466387554
episode is 659
Random_start:18
50897
0.605103263822
episode is 660
Random_start:83
50935
0.604877167542
episode is 661
Random_start:35
50962
0.604716572389
episode is 662
Normal_start:1
51009
0.604437121276
episode is 663
Random_start:39
51020
0.604371736789
episode is 664
Random_start:81
51059
0.604139977007
episode is 665
Ran